In [ ]:
import numpy as np
import os
import cv2
import dlib
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

# Configure logging
logging.basicConfig(level=logging.INFO)

def load_face_recognition_models():
    """
    Load the face detection, landmark predictor, and face recognition models.
    """
    face_detector = dlib.get_frontal_face_detector()
    shape_predictor = dlib.shape_predictor("Dlib_files/shape_predictor_68_face_landmarks.dat")
    face_encoder = dlib.face_recognition_model_v1("Dlib_files/dlib_face_recognition_resnet_model_v1.dat")
    return face_detector, shape_predictor, face_encoder


def extract_face_encodings(image, face_detector, shape_predictor, face_encoder):
    """
    Extract facial encodings from an image.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = face_detector(gray, 1)
    encodings = [np.array(face_encoder.compute_face_descriptor(image, shape_predictor(gray, rect))) for rect in rects]
    return encodings


def preprocess_images_and_labels(images_folder, face_detector, shape_predictor, face_encoder):
    """
    Preprocess images and extract encodings and labels.
    """
    encodings = []
    labels = []
    labels_dict ={}
    label = 0
    for root, dirs, files in os.walk(images_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)
                image = cv2.imread(image_path)
                if image is None:
                    logging.error(f"Failed to load image: {image_path}")
                    continue
                face_encs = extract_face_encodings(image, face_detector, shape_predictor, face_encoder)
                encodings.extend(face_encs)
                #labels.extend([os.path.basename(root)])
                labels.extend([os.path.basename(root)] * len(face_encs))
        labels_dict[label] = os.path.basename(root)
        label += 1
        #print("label dict", labels_dict)
        
    labels_dict.pop(0, None)       
    le = LabelEncoder()
    labels_encoded = le.fit_transform(labels)
    num_classes = len(le.classes_)
    print("clasess", num_classes)
    labels_encoded = to_categorical(labels_encoded, num_classes)  # Convert labels to one-hot encoding
    print("label dict", labels_dict)

    return np.array(encodings), np.array(labels_encoded), num_classes, labels_dict


def build_model(input_shape, num_classes):
    """
    Build the CNN model.
    """
    print("num clasees", num_classes)
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def evaluate_model(model, X_test, y_test, labels_dict):
    """
    Evaluate the trained CNN model.
    """
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)
    confidence_scores = np.max(y_pred_prob, axis=1)
    logging.info("Confidence Scores: %s", confidence_scores)
    accuracy = accuracy_score(y_true, y_pred)
    logging.info("Test Accuracy: %s", accuracy)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R^2 Score: {r2:.4f}")
    print("=" * 50)
    
    # Classification Report
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=labels_dict.values()))
    
    # Confusion Matrix
    print("Confusion Matrix:")
    plot_confusion_matrix(y_true, y_pred, labels_dict)
    


def loadModel():
    
    print("Training faces. It will take a few seconds. Please wait ...")

    # Load face recognition models
    face_detector, shape_predictor, face_encoder = load_face_recognition_models()

    # Preprocess images and labels
    encodings, labels, num_classes, labels_dict = preprocess_images_and_labels('dataset/images', face_detector, shape_predictor, face_encoder)

    # Preprocess data
    X_train, X_test, y_train, y_test = train_test_split(encodings, labels, test_size=0.2, random_state=42,
                                                        stratify=labels)

    # Build CNN model
    model = build_model(input_shape=X_train.shape[1:], num_classes=num_classes)

    # Train CNN model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

    # Plot Training History
    plot_training_history(history)
    
    # Evaluate CNN model
    evaluate_model(model, X_test, y_test, labels_dict)
       


def plot_training_history(history):
    """
    Plot training history (accuracy and loss).
    """
    plt.figure(figsize=(12, 6))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()



def plot_confusion_matrix(y_true, y_pred,labels_dict):
    """
    Plot confusion matrix using seaborn.
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=labels_dict.values(), yticklabels=labels_dict.values())
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()



loadModel()

